In [7]:
import pandas as pd
import numpy as np
from random import random
from CTL2.causal_tree_learn import CausalTree
from DGP.dgp4 import dgp_ate4_test
#from DGP.dgp6 import dgp_ate8

from MC.mc2 import MC_MSE_2
from MC.mc2_maxd import MC_MSE_2max
from utility.utility import transposer as tp
from utility.utility import writer
from utility.utility import reader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import pickle as pkl
#from notify_run import Notify 
#notify = Notify()
import datetime

ImportError: cannot import name 'dgp_ate4_test' from 'DGP.dgp4' (C:\Users\User\Desktop\codes1\CTL\DGP\dgp4.py)

In [3]:
def MC_ate4(n_train, n_test, var_e, reps, nomin_test):    
    #true ATE
    #ATE1 = np.ones(reps)*3
    #ATE2 = np.ones(reps)*1
    #ATE3 = np.ones(reps)*-2
    #ATE4 = np.ones(reps)*-3
    #predicted ate
    #ate1 = []
    #ate2 = []
    #ate3 = []
    #ate4 = []
    #TOTAL MSE, Bias, Var    
    tauhat = np.zeros((n_test, 1))
    x_test, y_test, treat_test = dgp_ate4_test(n_test, var_e)
    tau = np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0)),3, np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0)),1, np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0)),-2,-3)))
    #MONTE CARLO
    for i in range(reps):
        x_train, y_train, treat_train = dgp_ate4_test(n_train,var_e)     
        #true individual treatment effect
        #tau = np.where(((x1 >= 0) & (x2 >= 0)),2, np.where(((x1 >= 0) & (x2 < 0)),1, np.where(((x1 < 0) & (x2 >= 0)),-1,-2)))
        #predicted infividual treatment effect
        ctl_predict = MC_MSE_2max(x_train, x_test, y_train, y_test, treat_train, treat_test, nomin_test)       
        tauhat = np.append(tauhat, ctl_predict.reshape(-1,1), axis = 1)

    #variance and mean across monte carlo simulations for each observation. 10000x1 arrays
    ind_var = np.var(tauhat[:, 1:], axis = 1)
    ind_mean = np.mean(tauhat, axis = 1)
    
    total_var = np.sum(ind_var)/n_test
    total_bias = np.sum(np.square(np.subtract(ind_mean, tau)))/n_test

    
    return total_var, total_bias, total_var+total_bias

In [2]:
def MC_ate8(n_train, n_test, var_e, reps, nomin_test):
    #true ATE
    ATE1 = np.ones(reps)*5
    ATE2 = np.ones(reps)*-1
    ATE3 = np.ones(reps)*2.5
    ATE4 = np.ones(reps)*-3.5
    ATE5 = np.ones(reps)*3.5
    ATE6 = np.ones(reps)*-2.5
    ATE7 = np.ones(reps)*1
    ATE8 = np.ones(reps)*-5
    #predicted ate
    ate1 = []
    ate2 = []
    ate3 = []
    ate4 = []
    ate5 = []
    ate6 = []
    ate7 = []
    ate8 = []
    
    tauhat = np.zeros((n_test, 1))
    x_test, y_test, treat_test = dgp_ate8(n_test, var_e)
    tau = np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0)),5, 
                       np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0)), -1, 
                                np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0)), 2.5, 
                                         np.where(((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0)), -3.5, 
                                                  np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0)), 3.5, 
                                                           np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0)), -2.5, 
                                                                    np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0)), 1, -5)))))))
       
    for i in range(reps):
        x_train, y_train, treat_train = dgp_ate8(n_train, var_e)    
        #true individual treatment effect
         
        
        #predicted infividual treatment effect
        ctl_predict = MC_MSE_2(x_train, x_test, y_train, y_test, treat_train, treat_test, nomin_test)
        #predicted ATE
        ate1 = np.append(ate1, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0))]))
        ate2 = np.append(ate2, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0))]))
        ate3 = np.append(ate3, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0))]))
        ate4 = np.append(ate4, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0))]))
        ate5 = np.append(ate5, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0))]))
        ate6 = np.append(ate6, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0))]))
        ate7 = np.append(ate7, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0))]))
        ate8 = np.append(ate8, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0))]))

        #leafs = np.append(leafs, len(np.unique(ctl_predict)))
        #nn = int(len(ctl_predict))
        tauhat = np.append(tauhat, ctl_predict.reshape(-1,1), axis = 1)

    #ATE MSE's
    mean_ate = np.array([np.mean(ate1), np.mean(ate2), np.mean(ate3), np.mean(ate4), np.mean(ate5), np.mean(ate6), np.mean(ate7), np.mean(ate8)])    
    mse_ate = np.array([mse(ATE1, ate1), mse(ATE2, ate2),mse(ATE3, ate3), mse(ATE4, ate4),mse(ATE5, ate5), mse(ATE6, ate6), mse(ATE7, ate7), mse(ATE8, ate8)])
    bias_ate = np.array([np.sum(np.subtract(ATE1,ate1))/reps, np.sum(np.subtract(ATE2,ate2))/reps, np.sum(np.subtract(ATE3,ate3))/reps, np.sum(np.subtract(ATE4,ate4))/reps,
                         np.sum(np.subtract(ATE5,ate5))/reps, np.sum(np.subtract(ATE6,ate6))/reps, np.sum(np.subtract(ATE7,ate7))/reps, np.sum(np.subtract(ATE8,ate8))/reps])
    var_ate = np.array([np.var(ate1), np.var(ate2), np.var(ate3), np.var(ate4), np.var(ate5), np.var(ate6), np.var(ate7), np.var(ate8)])
    
    #MSE totals
    
    ind_var = np.var(tauhat[:, 1:], axis = 1)
    ind_mean = np.mean(tauhat, axis = 1)
    
    total_var = np.sum(ind_var)/n_test
    total_bias = np.sum(np.square(np.subtract(ind_mean, tau)))/n_test
    
    
    return mean_ate, mse_ate, bias_ate, var_ate, total_bias, total_var, total_bias+total_var

In [6]:
print("start time: ", str(datetime.datetime.now())[11:16])
test_sizes = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

MSE_ = []
bias_ = []
var_ = []

var_e = 0.01
n_test = 1000
n_train = 500
reps = 10

for size in test_sizes:
    bias, var, MSE = MC_ate4(n_train, n_test, var_e, reps, size)

    MSE_ = np.append(MSE_, MSE)
    bias_ = np.append(bias_, bias)
    var_ = np.append(var_, var)
    
#notify.send('Done running the simulations')
print("end time: ", str(datetime.datetime.now())[11:16])

start time:  11:45


NameError: name 'dgp_ate4_test' is not defined

#### True ATEs: 
* 5 
* -1
* 2.5
* -3.5
* 3.5 
* -2.5 
* 1 
* -5

In [ ]:
mean_ate_, mse_ate_, bias_ate_, var_ate_ = tp(mean_ate_, mse_ate_, bias_ate_, var_ate_, test_sizes, 8)

In [12]:
mean_ate_

NameError: name 'mean_ate_' is not defined

In [ ]:
mean_ate_ = mean_ate_.reshape(len(test_sizes),8).transpose()
mse_ate_ = mse_ate_.reshape(len(test_sizes),8).transpose()
bias_ate_ = bias_ate_.reshape(len(test_sizes),8).transpose()
var_ate_ = var_ate_.reshape(len(test_sizes),8).transpose()
df = pd.DataFrame([test_sizes, mean_ate_[0], mean_ate_[1],mean_ate_[2], mean_ate_[3], mse_ate_[0], mse_ate_[1],mse_ate_[2], mse_ate_[3], bias_ate_[0], bias_ate_[1], bias_ate_[2], bias_ate_[3], var_ate_[0], var_ate_[1],var_ate_[2], var_ate_[3], MSE_, bias_, var_]).transpose().set_index(0)
df.columns=['Mean1', 'Mean2', 'Mean3', 'Mean4', 'MSE1', 'MSE2','MSE3', 'MSE4', 'BIAS1', 'BIAS2','BIAS3', 'BIAS4', 'VAR1', 'VAR2','VAR3', 'VAR4', 'MSE_TOTAL', 'MSE_T_BIAS', 'MSE_T_VAR']